<a href="https://colab.research.google.com/github/chacha86/pythonai/blob/main/%EC%9B%B9%EC%8A%A4%ED%81%AC%EB%9E%A9%ED%95%912.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:

import requests

## 브라우저인 척 하기위해 헤더 정보에 브라우저 정보 포함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 해당 url로 요청 보내서 html 문서 받아오기
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)
    
## html 문서 가져오기
html_doc = r.text

## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')


## css 선택자를 사용하기 위한 함수 =>
### 1. select()  => 결과가 list
### 2. select_one() => 결과가 1개

## 텍스트와 속성값 얻기
### 1. 태그.text
### 2. 태그[key]
###    - key = value

# 1. 언론사 이름과 링크 가져오기
boxes = soup.select('.rankingnews_box')

journal_list = []

for box in boxes :

  atag = box.select_one('.rankingnews_box_head')
  strong = box.select_one('.rankingnews_name')

  name = strong.text
  link = atag['href']

  journal = {
      'name' : name,
      'link' : link
  }

  journal_list.append(journal)

## 결과 중 YTN만 가져오기
target = None
for journal in journal_list :
  if journal['name'] == 'SBS' :
    target = journal
    break


# 결과 확인
target


{'name': 'SBS',
 'link': 'https://media.naver.com/press/055/ranking?type=popular'}

In [42]:
# url을 넘기면 해당 url 페이지의 html을 파싱해서 soup를 만들어주는 함수
def get_page(url) :
  r2 = requests.get(url, headers=headers)

  html_doc2 = r2.text

  ## BeautifulSoup 객체에 넘겨 파싱하기(정보를 태그 단위로 쪼개기)
  from bs4 import BeautifulSoup
  soup = BeautifulSoup(html_doc2, 'html.parser')

  return soup



## soup를 넘겨주면 해당 soup에서 랭킹 뉴스 20개를 가져오는 함수
def get_total_news(page) : 
  
  total_news_list = []
  groups = page.select('.press_ranking_box')
  for group in groups :
    news_list_tmp = get_news_by_group(group)
    total_news_list.extend(news_list_tmp)

  return total_news_list

## 뉴스 그룹을 넘기면 해당 그룹의 뉴스를 모두 가져와 합쳐주는 함수
def get_news_by_group(group) :
  news_list = []
  thumb_list = group.select('.as_thumb')
  for news in thumb_list :
    atag = news.select_one('a')
    link = atag['href']
    title = atag.select_one('.list_title').text
    
    news = {
        'title' : title,
        'link' : link
    }
    news_list.append(news)

  return news_list


## YTN의 url
url = target['link']

## YTN의 언론사 페이지 정보를 가진 soup
soup = get_page(url)

## YTN의 랭킹뉴스 20개
result = get_total_news(soup)

## 결과 확인
result

In [43]:
import json

## 파이썬 오브젝트를 json 형식으로 저장하는 함수
def save_to_json(filepath, data) :
  with open(filepath,'w') as f :  
    json_str = json.dumps(data, ensure_ascii=False) 
    f.write(json_str)




In [44]:
## json 형식의 text를 파이썬 오브젝트로 불러오는 함수
def load_to_json(filepath) :
  with open(filepath, 'r') as f:
    json_str = f.read()
    return json.loads(json_str)

  


In [ ]:
# 3. 랭킹뉴스를 날짜별로

# 최근 10일 동안의 뉴스 가져오기

## 특정 구간의 날짜 생성하기
from datetime import datetime, timedelta

def date_range(start, end):
    start = datetime.strptime(start, "%Y%m%d")
    end = datetime.strptime(end, "%Y%m%d")
    dates = [(start + timedelta(days=i)).strftime("%Y%m%d") for i in range((end-start).days+1)]
    return dates

dates = date_range("20210101", "20210109")




## 현재 날짜 가져오기
from datetime import datetime, timedelta
today = datetime.today()            


def get_dates(time) :
  ## 날짜 형식을 내가 원하는 대로 바꾸기
  formated_today = today.strftime('%Y%m%d')
  print(formated_today)


  ## 날짜 계산하기

  ### 오늘 날짜로 부터 10일 전
  start_day = today - timedelta(days=time)
  end_day = today

  ### 오늘 날짜로 부터 10일 전부터 오늘 날짜까지의 날짜데이터 생성
  dates = date_range(start_day.strftime('%Y%m%d'), end_day.strftime('%Y%m%d'))

  return dates

### 결과 확인
dates = get_dates(5)

dates

In [52]:
# 언론사 링크를 넘기면 최근 n일치의 랭킹 뉴스 링크를 반환하는 함수
def get_news_link_by_time(url, dates) :
  
  link_list = []
  url = url + '&date=' 

  for date in dates :
    link = url + date
    link_list.append(link)

  return link_list


## YTN 언론사 링크
journal_url1 = target['link']

## YTN 언론사의 10일치 랭킹 뉴스 링크
news_link_by_time = get_news_link_by_time(journal_url1, dates)

## 결과 확인
news_link_by_time

['https://media.naver.com/press/052/ranking?type=popular&date=20230414',
 'https://media.naver.com/press/052/ranking?type=popular&date=20230415',
 'https://media.naver.com/press/052/ranking?type=popular&date=20230416',
 'https://media.naver.com/press/052/ranking?type=popular&date=20230417',
 'https://media.naver.com/press/052/ranking?type=popular&date=20230418',
 'https://media.naver.com/press/052/ranking?type=popular&date=20230419']

In [ ]:

# 작성된 함수들을 이용해 10일치 랭킹뉴스 모두 가져와 json파일로 저장하기

## news_link_by_time -> 20230409 ~ 20230419 까지의 YTN 랭킹 뉴스 링크
## get_page(link) -> link의 html 문서를 파싱해서 soup를 반환
## get_total_news(soup) -> soup에서 랭킹 뉴스를 스크랩하는 함수
## dates => 20230409 ~ 20230419 날짜 정보


for date, news_link in zip(dates, news_link_by_time) : # 10일치의 뉴스 링크를 for문으로 하나씩 빼서
  soup = get_page(news_link) # 해당 링크의 페이지를 soup형태로 얻고
  news_list = get_total_news(soup) # 20개의 랭킹뉴스를 얻어오는 함수에 넣어 
  save_to_json(f'YTN_{date}.json', news_list) # 결과를 파일로 저장한다.



In [57]:
# 실습 -> 3개의 언론사 (SBS, MBC, KBS)의 랭킹 뉴스를 최근 5일치 가져와서 각각 저장.
# 언론사별로 폴더를 나눠서 저장

def save_news_by_time(time, journal) :

  dates = get_dates(time)
  jname = journal['name']
  jlink = journal['link']

  news_link_by_time = get_news_link_by_time(jlink, dates)

  for date, news_link in zip(dates, news_link_by_time) : # 10일치의 뉴스 링크를 for문으로 하나씩 빼서
    soup = get_page(news_link) # 해당 링크의 페이지를 soup형태로 얻고
    news_list = get_total_news(soup) # 20개의 랭킹뉴스를 얻어오는 함수에 넣어 
    save_to_json(f'{jname}/{jname}_{date}.json', news_list) # 결과를 파일로 저장한다.

In [ ]:
# 실습 -> 3개의 언론사 (SBS, MBC, KBS)의 랭킹 뉴스를 최근 5일치 가져와서 각각 저장.
# 언론사별로 폴더를 나눠서 저장
save_news_by_time(5, target)





In [ ]:
# 
  



In [ ]:
# 4. 특정 뉴스의 댓글 통계 정보